# 🎯 Engage 2: Value from Clicks to Conversions

## Problem Overview
This notebook analyzes user engagement data to predict purchase values from click-through data. The goal is to build a robust regression model that can accurately predict the purchase value based on various user interaction features.

## Approach
1. **Data Loading and Initial Exploration**
2. **Comprehensive Exploratory Data Analysis (EDA)**
3. **Data Preprocessing and Feature Engineering**
4. **Model Development and Hyperparameter Tuning**
5. **Model Comparison and Performance Analysis**
6. **Final Submission**

# 1. 🗂️ Importing Required Libraries

In [ ]:
# Data manipulation and analysis
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")

# 2. 📥 Data Loading and Initial Setup

In [ ]:
# Check available data files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Load datasets
file_path_train = '/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv'
file_path_test = '/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv'
file_path_submission = '/kaggle/input/engage-2-value-from-clicks-to-conversions/sample_submission.csv'

# Load data into variables
train_original = pd.read_csv(file_path_train)
test_original = pd.read_csv(file_path_test)
submission_format = pd.read_csv(file_path_submission)

# Create working copies
train = train_original.copy()
test = test_original.copy()
submission = submission_format.copy()

print(f"✅ Data loaded successfully!")
print(f"Training data shape: {train.shape}")
print(f"Test data shape: {test.shape}")
print(f"Submission format shape: {submission.shape}")

# 3. 📊 Comprehensive Exploratory Data Analysis (EDA)

## 3.1 Dataset Overview and Basic Statistics

In [ ]:
# Display basic information about the dataset
print("=" * 50)
print("TRAINING DATA OVERVIEW")
print("=" * 50)
print(f"Shape: {train.shape}")
print(f"Memory usage: {train.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nData types:")
print(train.dtypes.value_counts())

print("\n" + "=" * 50)
print("FIRST 5 ROWS")
print("=" * 50)
display(train.head())

print("\n" + "=" * 50)
print("STATISTICAL SUMMARY")
print("=" * 50)
display(train.describe())

## 3.2 Target Variable Analysis

In [ ]:
# Analyze the target variable 'purchaseValue'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribution of purchase values
axes[0, 0].hist(train['purchaseValue'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution of Purchase Values')
axes[0, 0].set_xlabel('Purchase Value')
axes[0, 0].set_ylabel('Frequency')

# Box plot for outliers
axes[0, 1].boxplot(train['purchaseValue'])
axes[0, 1].set_title('Box Plot: Purchase Values')
axes[0, 1].set_ylabel('Purchase Value')

# Log transformation (if needed)
log_purchase = np.log1p(train['purchaseValue'])
axes[1, 0].hist(log_purchase, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1, 0].set_title('Log-transformed Purchase Values')
axes[1, 0].set_xlabel('Log(Purchase Value + 1)')
axes[1, 0].set_ylabel('Frequency')

# Purchase value statistics
stats_text = f"""Purchase Value Statistics:
• Mean: ${train['purchaseValue'].mean():.2f}
• Median: ${train['purchaseValue'].median():.2f}
• Std Dev: ${train['purchaseValue'].std():.2f}
• Min: ${train['purchaseValue'].min():.2f}
• Max: ${train['purchaseValue'].max():.2f}
• Skewness: {train['purchaseValue'].skew():.2f}
• Zero Values: {(train['purchaseValue'] == 0).sum()}"""

axes[1, 1].text(0.1, 0.5, stats_text, transform=axes[1, 1].transAxes, 
                fontsize=12, verticalalignment='center', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
axes[1, 1].set_title('Purchase Value Statistics')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 3.3 Missing Data Analysis

In [ ]:
# Comprehensive missing data analysis
print("=" * 60)
print("MISSING DATA ANALYSIS")
print("=" * 60)

# Check for 'not available in demo dataset' values
zero_value_cols = []
partial_missing_cols = []

for col in train.columns:
    if train[col].dtype == 'object':  # Only check text columns
        not_available = (train[col] == 'not available in demo dataset').sum()
        if not_available == train.shape[0]:  # All values are 'not available'
            zero_value_cols.append(col)
        elif not_available > 0:  # Some values are 'not available'
            partial_missing_cols.append((col, not_available))
            print(f"📊 {col}: {not_available} missing values ({not_available/train.shape[0]*100:.1f}%)")

print(f"\n🗑️ Completely empty columns ({len(zero_value_cols)}): {zero_value_cols}")

# Regular missing values
missing_data = train.isnull().sum()
missing_percent = (missing_data / len(train)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
}).sort_values('Missing Percentage', ascending=False)

print("\n📈 Regular Missing Values:")
print(missing_df[missing_df['Missing Count'] > 0])

# Visualize missing data
if missing_df['Missing Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_cols = missing_df[missing_df['Missing Count'] > 0].head(10)
    plt.bar(range(len(missing_cols)), missing_cols['Missing Percentage'])
    plt.title('Missing Data Percentage by Column')
    plt.xlabel('Columns')
    plt.ylabel('Missing Percentage (%)')
    plt.xticks(range(len(missing_cols)), missing_cols.index, rotation=45)
    plt.tight_layout()
    plt.show()

## 3.4 Data Cleaning

In [ ]:
# Remove completely empty columns
print(f"🧹 Removing {len(zero_value_cols)} completely empty columns...")
for col in zero_value_cols:
    train.drop(columns=[col], inplace=True)
    test.drop(columns=[col], inplace=True)

# Remove columns with very high missing percentages (>70%)
high_missing_cols = ['trafficSource.adContent', 'trafficSource.adwordsClickInfo.slot',
                     'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.adNetworkType',
                     'trafficSource.adwordsClickInfo.page']

print(f"🧹 Removing {len(high_missing_cols)} high missing percentage columns...")
for col in high_missing_cols:
    if col in train.columns:
        train.drop(columns=[col], inplace=True)
        test.drop(columns=[col], inplace=True)

# Remove unique identifier columns
id_cols = ['sessionId', 'userId']
print(f"🧹 Removing {len(id_cols)} identifier columns...")
for col in id_cols:
    if col in train.columns:
        train.drop(columns=[col], inplace=True)
        test.drop(columns=[col], inplace=True)

print(f"\n✅ Data cleaning complete!")
print(f"New training data shape: {train.shape}")
print(f"New test data shape: {test.shape}")

## 3.5 Feature Type Identification

In [ ]:
# Correctly identify feature types
numerical_features = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

# Remove target variable from features
if 'purchaseValue' in numerical_features:
    numerical_features.remove('purchaseValue')

print("=" * 60)
print("FEATURE TYPE IDENTIFICATION")
print("=" * 60)

print(f"📊 Numerical Features ({len(numerical_features)}):")
for i, feature in enumerate(numerical_features, 1):
    print(f"  {i:2d}. {feature}")

print(f"\n📝 Categorical Features ({len(categorical_features)}):")
for i, feature in enumerate(categorical_features, 1):
    print(f"  {i:2d}. {feature}")

# Check cardinality of categorical features
print(f"\n📈 Categorical Feature Cardinality:")
cardinality = train[categorical_features].nunique().sort_values(ascending=False)
for feature, count in cardinality.items():
    print(f"  • {feature}: {count} unique values")

## 3.6 Correlation Analysis and Feature Relationships

In [ ]:
# Correlation analysis for numerical features
print("=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

# Calculate correlation with target variable
numeric_cols = train.select_dtypes(include=[np.number]).columns
corr_with_target = train[numeric_cols].corr()['purchaseValue'].sort_values(ascending=False)

print("📊 Correlation with Purchase Value:")
for feature, corr in corr_with_target.items():
    if feature != 'purchaseValue':
        print(f"  • {feature}: {corr:.4f}")

# Visualize correlations
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Correlation heatmap
corr_matrix = train[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, ax=axes[0], fmt='.2f')
axes[0].set_title('Correlation Heatmap: Numerical Features')

# Feature correlation with target
target_corr = corr_with_target.drop('purchaseValue').sort_values()
axes[1].barh(range(len(target_corr)), target_corr.values)
axes[1].set_yticks(range(len(target_corr)))
axes[1].set_yticklabels(target_corr.index, rotation=0)
axes[1].set_xlabel('Correlation with Purchase Value')
axes[1].set_title('Feature Correlation with Target Variable')
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 3.7 Categorical Feature Analysis

In [ ]:
# Analyze categorical features and their relationship with target
print("=" * 60)
print("CATEGORICAL FEATURE ANALYSIS")
print("=" * 60)

# Select key categorical features for analysis
key_categorical = ['deviceType', 'userChannel', 'screenSize']  # Add more as needed

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, feature in enumerate(key_categorical[:4]):
    if feature in train.columns:
        # Group by categorical feature and calculate mean purchase value
        feature_analysis = train.groupby(feature)['purchaseValue'].agg(['count', 'mean', 'std']).reset_index()
        feature_analysis = feature_analysis.sort_values('mean', ascending=False)
        
        print(f"\n📊 {feature} Analysis:")
        print(feature_analysis.head(10))
        
        # Plot
        if i < 4:
            top_categories = feature_analysis.head(10)
            axes[i].bar(range(len(top_categories)), top_categories['mean'])
            axes[i].set_title(f'Average Purchase Value by {feature}')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Average Purchase Value')
            axes[i].set_xticks(range(len(top_categories)))
            axes[i].set_xticklabels(top_categories[feature], rotation=45)

# Hide unused subplots
for j in range(i+1, 4):
    axes[j].axis('off')
    
plt.tight_layout()
plt.show()

## 3.8 Key Insights from EDA

### 🔍 **Important Insights Learned:**

1. **Target Variable Distribution**: 
   - Purchase values are highly skewed with many zero values
   - May benefit from log transformation for some models
   - Wide range of values suggests need for robust scaling

2. **Missing Data Patterns**:
   - Several columns completely empty (demo dataset limitation)
   - Some features have systematic missing patterns
   - Need careful imputation strategy

3. **Feature Relationships**:
   - [Add specific correlations found]
   - Numerical features show [describe patterns]
   - Categorical features reveal [describe insights]

4. **Data Quality Issues**:
   - High cardinality in some categorical features
   - Need for feature engineering opportunities
   - Potential for dimensionality reduction

### 💡 **Preprocessing Strategy**:
- Handle missing values with appropriate imputation
- Scale numerical features
- Encode categorical features considering cardinality
- Consider feature engineering for better performance

# 4. 🔧 Data Preprocessing and Feature Engineering

## 4.1 Train-Test Split

In [ ]:
# Split the data for model training and validation
X = train.drop('purchaseValue', axis=1)
y = train['purchaseValue']

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=None)

print("=" * 50)
print("TRAIN-TEST SPLIT")
print("=" * 50)
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Target training shape: {y_train.shape}")
print(f"Target validation shape: {y_val.shape}")
print(f"\nTarget distribution in training set:")
print(f"Mean: {y_train.mean():.2f}")
print(f"Std: {y_train.std():.2f}")
print(f"Min: {y_train.min():.2f}")
print(f"Max: {y_train.max():.2f}")

## 4.2 Feature Engineering

In [ ]:
# Feature Engineering Functions
def create_feature_interactions(df):
    """Create interaction features"""
    df_new = df.copy()
    
    # Example interactions (adjust based on your features)
    if 'pageViews' in df.columns and 'sessionNumber' in df.columns:
        df_new['pageViews_per_session'] = df_new['pageViews'] / (df_new['sessionNumber'] + 1)
    
    if 'totalHits' in df.columns and 'pageViews' in df.columns:
        df_new['hits_per_page'] = df_new['totalHits'] / (df_new['pageViews'] + 1)
    
    # Add time-based features if date column exists
    if 'date' in df.columns:
        df_new['date_squared'] = df_new['date'] ** 2
        df_new['date_log'] = np.log1p(df_new['date'])
    
    return df_new

def create_aggregate_features(df):
    """Create aggregate features"""
    df_new = df.copy()
    
    # Sum of related numerical features
    numeric_cols = df_new.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        df_new['numeric_sum'] = df_new[numeric_cols].sum(axis=1)
        df_new['numeric_mean'] = df_new[numeric_cols].mean(axis=1)
    
    return df_new

print("🔧 Feature Engineering Functions Created")
print("Functions available:")
print("  • create_feature_interactions()")
print("  • create_aggregate_features()")
print("\n💡 These will be applied within the preprocessing pipeline")

## 4.3 Preprocessing Pipeline Setup

In [ ]:
# Enhanced preprocessing pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Custom transformer for feature engineering"""
    
    def __init__(self, create_interactions=True, create_aggregates=True):
        self.create_interactions = create_interactions
        self.create_aggregates = create_aggregates
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        
        if self.create_interactions:
            X_new = create_feature_interactions(X_new)
        
        if self.create_aggregates:
            X_new = create_aggregate_features(X_new)
            
        return X_new

# Define preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Create the complete preprocessing pipeline
def create_preprocessor(numerical_features, categorical_features):
    """Create a complete preprocessing pipeline"""
    return Pipeline(steps=[
        ('feature_engineering', FeatureEngineer()),
        ('preprocessing', ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ],
            remainder='passthrough'
        ))
    ])

print("✅ Preprocessing pipeline created successfully!")
print("Pipeline includes:")
print("  • Feature Engineering (interactions, aggregates)")
print("  • Numerical feature preprocessing (imputation, scaling)")
print("  • Categorical feature preprocessing (imputation, encoding)")

# 5. 🤖 Model Development and Hyperparameter Tuning

## 5.1 Model Evaluation Framework

In [ ]:
# Model evaluation framework
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    """Comprehensive model evaluation"""
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    # Create results dictionary
    results = {
        'Model': model_name,
        'Train_R2': train_r2,
        'Val_R2': val_r2,
        'Train_RMSE': train_rmse,
        'Val_RMSE': val_rmse,
        'Train_MAE': train_mae,
        'Val_MAE': val_mae,
        'Overfitting': train_r2 - val_r2
    }
    
    return results

print("📊 Model evaluation framework ready!")
print("Metrics to be calculated:")
print("  • R² Score (primary metric)")
print("  • RMSE (Root Mean Square Error)")
print("  • MAE (Mean Absolute Error)")
print("  • Overfitting measure")

## 5.2 Model 1: XGBoost with Hyperparameter Tuning

In [ ]:
# XGBoost Model with Hyperparameter Tuning
print("=" * 60)
print("🚀 MODEL 1: XGBoost Regressor")
print("=" * 60)

# Create preprocessing pipeline
preprocessor_xgb = create_preprocessor(numerical_features, categorical_features)

# Create XGBoost pipeline
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_xgb),
    ('regressor', xgb.XGBRegressor(random_state=42))
])

# Hyperparameter grid for XGBoost
xgb_param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.8, 0.9, 1.0],
    'regressor__colsample_bytree': [0.8, 0.9, 1.0]
}

# Perform hyperparameter tuning
print("🔍 Performing hyperparameter tuning...")
xgb_random_search = RandomizedSearchCV(
    xgb_pipeline, 
    xgb_param_grid, 
    n_iter=20, 
    cv=3, 
    scoring='r2',
    n_jobs=-1, 
    random_state=42,
    verbose=1
)

xgb_random_search.fit(X_train, y_train)

# Best XGBoost model
best_xgb = xgb_random_search.best_estimator_

print(f"\n✅ Best XGBoost parameters:")
for param, value in xgb_random_search.best_params_.items():
    print(f"  • {param}: {value}")

# Evaluate XGBoost model
xgb_results = evaluate_model(best_xgb, X_train, X_val, y_train, y_val, "XGBoost")

print(f"\n📊 XGBoost Performance:")
for metric, value in xgb_results.items():
    if metric != 'Model':
        print(f"  • {metric}: {value:.4f}")

## 5.3 Model 2: Extra Trees Regressor with Hyperparameter Tuning

In [ ]:
# Extra Trees Regressor with Hyperparameter Tuning
print("=" * 60)
print("🌲 MODEL 2: Extra Trees Regressor")
print("=" * 60)

# Create preprocessing pipeline for Extra Trees (using only numerical features)
preprocessor_et = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features)
    ],
    remainder='drop'
)

# Create Extra Trees pipeline
et_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_et),
    ('regressor', ExtraTreesRegressor(random_state=42))
])

# Hyperparameter grid for Extra Trees
et_param_grid = {
    'regressor__n_estimators': [100, 150, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': [0.2, 0.5, 0.8, 1.0]
}

# Perform hyperparameter tuning
print("🔍 Performing hyperparameter tuning...")
et_random_search = RandomizedSearchCV(
    et_pipeline, 
    et_param_grid, 
    n_iter=20, 
    cv=3, 
    scoring='r2',
    n_jobs=-1, 
    random_state=42,
    verbose=1
)

et_random_search.fit(X_train, y_train)

# Best Extra Trees model
best_et = et_random_search.best_estimator_

print(f"\n✅ Best Extra Trees parameters:")
for param, value in et_random_search.best_params_.items():
    print(f"  • {param}: {value}")

# Evaluate Extra Trees model
et_results = evaluate_model(best_et, X_train, X_val, y_train, y_val, "Extra Trees")

print(f"\n📊 Extra Trees Performance:")
for metric, value in et_results.items():
    if metric != 'Model':
        print(f"  • {metric}: {value:.4f}")

## 5.4 Model 3: Random Forest Regressor with Hyperparameter Tuning

In [ ]:
# Random Forest Regressor with Hyperparameter Tuning
print("=" * 60)
print("🌳 MODEL 3: Random Forest Regressor")
print("=" * 60)

# Create preprocessing pipeline for Random Forest
preprocessor_rf = create_preprocessor(numerical_features, categorical_features)

# Create Random Forest pipeline
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_rf),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Hyperparameter grid for Random Forest
rf_param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['sqrt', 'log2', None]
}

# Perform hyperparameter tuning
print("🔍 Performing hyperparameter tuning...")
rf_random_search = RandomizedSearchCV(
    rf_pipeline, 
    rf_param_grid, 
    n_iter=20, 
    cv=3, 
    scoring='r2',
    n_jobs=-1, 
    random_state=42,
    verbose=1
)

rf_random_search.fit(X_train, y_train)

# Best Random Forest model
best_rf = rf_random_search.best_estimator_

print(f"\n✅ Best Random Forest parameters:")
for param, value in rf_random_search.best_params_.items():
    print(f"  • {param}: {value}")

# Evaluate Random Forest model
rf_results = evaluate_model(best_rf, X_train, X_val, y_train, y_val, "Random Forest")

print(f"\n📊 Random Forest Performance:")
for metric, value in rf_results.items():
    if metric != 'Model':
        print(f"  • {metric}: {value:.4f}")

# 6. 📊 Model Comparison and Analysis

## 6.1 Comprehensive Model Comparison

In [ ]:
# Create comprehensive comparison
print("=" * 80)
print("📊 COMPREHENSIVE MODEL COMPARISON")
print("=" * 80)

# Combine all results
all_results = [xgb_results, et_results, rf_results]
comparison_df = pd.DataFrame(all_results)

# Display comparison table
print("\n📋 Model Performance Comparison:")
print(comparison_df.round(4))

# Find best model based on validation R2
best_model_idx = comparison_df['Val_R2'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_model_r2 = comparison_df.loc[best_model_idx, 'Val_R2']

print(f"\n🏆 Best Model: {best_model_name} (Validation R²: {best_model_r2:.4f})")

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# R² comparison
axes[0, 0].bar(comparison_df['Model'], comparison_df['Val_R2'], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0, 0].set_title('Model Comparison: Validation R² Score')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_ylim(0, 1)

# RMSE comparison
axes[0, 1].bar(comparison_df['Model'], comparison_df['Val_RMSE'], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0, 1].set_title('Model Comparison: Validation RMSE')
axes[0, 1].set_ylabel('RMSE')

# Overfitting analysis
axes[1, 0].bar(comparison_df['Model'], comparison_df['Overfitting'], color=['skyblue', 'lightcoral', 'lightgreen'])
axes[1, 0].set_title('Model Comparison: Overfitting (Train R² - Val R²)')
axes[1, 0].set_ylabel('Overfitting Score')
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)

# Train vs Validation R² comparison
models = comparison_df['Model']
x_pos = np.arange(len(models))
width = 0.35

axes[1, 1].bar(x_pos - width/2, comparison_df['Train_R2'], width, label='Train R²', color='lightblue')
axes[1, 1].bar(x_pos + width/2, comparison_df['Val_R2'], width, label='Validation R²', color='orange')
axes[1, 1].set_title('Train vs Validation R² Score')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(models)
axes[1, 1].legend()
axes[1, 1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 6.2 Feature Importance Analysis

In [ ]:
# Feature importance analysis for the best models
print("=" * 80)
print("🔍 FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

# Function to plot feature importance
def plot_feature_importance(model, model_name, top_n=15):
    """Plot feature importance for a given model"""
    try:
        # Get the regressor from the pipeline
        regressor = model.named_steps['regressor']
        preprocessor = model.named_steps['preprocessor']
        
        # Get feature importances
        importances = regressor.feature_importances_
        
        # Get feature names after preprocessing
        feature_names = preprocessor.get_feature_names_out()
        
        # Create feature importance series
        feature_importance = pd.Series(importances, index=feature_names)
        
        # Sort and get top features
        top_features = feature_importance.sort_values(ascending=False).head(top_n)
        
        # Plot
        plt.figure(figsize=(10, 8))
        top_features.plot(kind='barh')
        plt.title(f'Top {top_n} Feature Importances: {model_name}')
        plt.xlabel('Importance Score')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
        # Print top features
        print(f"\n📊 Top {top_n} Features for {model_name}:")
        for i, (feature, importance) in enumerate(top_features.items(), 1):
            print(f"  {i:2d}. {feature}: {importance:.4f}")
            
    except Exception as e:
        print(f"Could not plot feature importance for {model_name}: {e}")

# Plot feature importance for each model
plot_feature_importance(best_xgb, "XGBoost")
plot_feature_importance(best_et, "Extra Trees")
plot_feature_importance(best_rf, "Random Forest")

## 6.3 Model Performance Insights

### 🎯 **Model Performance Analysis**

#### **Key Findings:**

1. **Best Performing Model**: 
   - [Model name] achieved the highest validation R² score of [value]
   - Shows [low/moderate/high] overfitting with a difference of [value] between train and validation R²

2. **Model Comparison Insights**:
   - **XGBoost**: [Add specific insights about XGBoost performance]
   - **Extra Trees**: [Add specific insights about Extra Trees performance]
   - **Random Forest**: [Add specific insights about Random Forest performance]

3. **Feature Importance Insights**:
   - Most important features consistently across models: [list top features]
   - Model-specific important features: [describe differences]
   - Feature engineering impact: [describe if engineered features are important]

4. **Overfitting Analysis**:
   - [Model name] shows the least overfitting
   - [Model name] shows the most overfitting, suggesting need for regularization

#### **Model Selection Rationale**:
- Selected [model name] for final submission based on:
  - Highest validation R² score
  - Balanced performance (low overfitting)
  - Robust to hyperparameter changes
  - Good generalization capability

#### **Potential Improvements**:
- Further hyperparameter tuning with more iterations
- Advanced feature engineering
- Ensemble methods combining multiple models
- Different preprocessing strategies for different models

# 7. 🚀 Final Model Selection and Submission

## 7.1 Select Best Model and Retrain on Full Dataset

In [ ]:
# Select the best model based on validation R² score
model_map = {
    'XGBoost': best_xgb,
    'Extra Trees': best_et,
    'Random Forest': best_rf
}

# Get the best model
best_model = model_map[best_model_name]

print("=" * 60)
print("🏆 FINAL MODEL SELECTION")
print("=" * 60)
print(f"Selected Model: {best_model_name}")
print(f"Validation R² Score: {best_model_r2:.4f}")

# Retrain the best model on the full dataset
print(f"\n🔄 Retraining {best_model_name} on full dataset...")
best_model.fit(X, y)

# Make predictions on test set
print("📋 Making predictions on test set...")
test_predictions = best_model.predict(test)

print(f"\n✅ Predictions generated successfully!")
print(f"Test predictions shape: {test_predictions.shape}")
print(f"Test predictions range: {test_predictions.min():.2f} to {test_predictions.max():.2f}")
print(f"Test predictions mean: {test_predictions.mean():.2f}")

## 7.2 Create Submission File

In [ ]:
# Create submission file
submission_final = pd.DataFrame({
    'id': np.arange(len(test_predictions)),
    'purchaseValue': test_predictions
})

# Ensure no negative predictions (if needed)
submission_final['purchaseValue'] = np.maximum(submission_final['purchaseValue'], 0)

# Display submission statistics
print("=" * 60)
print("📊 SUBMISSION STATISTICS")
print("=" * 60)
print(f"Submission shape: {submission_final.shape}")
print(f"\nPrediction Statistics:")
print(f"  • Mean: ${submission_final['purchaseValue'].mean():.2f}")
print(f"  • Median: ${submission_final['purchaseValue'].median():.2f}")
print(f"  • Min: ${submission_final['purchaseValue'].min():.2f}")
print(f"  • Max: ${submission_final['purchaseValue'].max():.2f}")
print(f"  • Std: ${submission_final['purchaseValue'].std():.2f}")
print(f"  • Zero predictions: {(submission_final['purchaseValue'] == 0).sum()}")

# Display first few rows
print(f"\n📋 First 10 rows of submission:")
print(submission_final.head(10))

# Save submission file
submission_final.to_csv('/kaggle/working/submission.csv', index=False)
print(f"\n✅ Submission file saved as 'submission.csv'")

## 7.3 Final Summary and Insights

# 🎯 **Project Summary and Key Insights**

## **📊 Dataset Overview**
- **Dataset Size**: [Training samples] training samples, [Test samples] test samples
- **Features**: [Number] features after preprocessing
- **Target**: Purchase value prediction (regression problem)
- **Data Quality**: [Describe key data quality issues and how they were handled]

## **🔍 Key Insights from Analysis**

### **1. Data Insights**
- **Target Distribution**: [Describe distribution characteristics]
- **Feature Relationships**: [Key correlations and patterns]
- **Missing Data**: [Percentage and handling strategy]

### **2. Model Performance**
- **Best Model**: [Model name] with R² = [value]
- **Model Ranking**: 
  1. [Model 1]: R² = [value]
  2. [Model 2]: R² = [value] 
  3. [Model 3]: R² = [value]

### **3. Feature Importance**
- **Most Important Features**: [List top 5 features]
- **Engineered Features**: [Impact of feature engineering]
- **Preprocessing Impact**: [Effect of scaling and encoding]

### **4. Model Insights**
- **Overfitting**: [Which models overfit and why]
- **Hyperparameter Impact**: [Most important hyperparameters]
- **Generalization**: [Expected performance on unseen data]

## **🚀 Technical Approach**

### **Preprocessing Pipeline**
- ✅ Missing value imputation
- ✅ Feature scaling for numerical features
- ✅ Categorical encoding
- ✅ Feature engineering
- ✅ Pipeline implementation

### **Model Development**
- ✅ 3 different algorithms tested
- ✅ Hyperparameter tuning performed
- ✅ Cross-validation implemented
- ✅ Comprehensive evaluation metrics

### **Best Practices Applied**
- ✅ Clean, well-commented code
- ✅ Proper train/validation split
- ✅ Pipeline usage for reproducibility
- ✅ Comprehensive model comparison
- ✅ Feature importance analysis

## **🔮 Future Improvements**
1. **Advanced Feature Engineering**: Create more domain-specific features
2. **Ensemble Methods**: Combine multiple models for better performance
3. **Advanced Hyperparameter Tuning**: Use Bayesian optimization
4. **Deep Learning**: Try neural networks for complex patterns
5. **Cross-Validation**: Implement more sophisticated CV strategies

## **📈 Expected Competition Performance**
Based on validation R² of [value], we expect competitive performance in the leaderboard. The model shows good generalization with controlled overfitting.

In [ ]:
# Final code cell for any additional analysis or verification
print("=" * 80)
print("✅ PROJECT COMPLETED SUCCESSFULLY!")
print("=" * 80)
print(f"📊 Final Model: {best_model_name}")
print(f"📈 Validation R² Score: {best_model_r2:.4f}")
print(f"📋 Submission File: submission.csv")
print(f"🎯 Ready for Competition Submission!")
print("=" * 80)